In [ ]:
import os

import torch
import torch.nn as nn

from dataloaders import *
from scene_net import *
from prune_utils import *

import torch.nn.utils.prune as prune

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
dataset = 'nyuv2_3'
ration = 0.7
task = 'T1+T3'
method = 'LTH'

In [ ]:
if dataset == "nyuv2_3":
    from config_nyuv2_3task import Config
    config = Config()
elif dataset == "cityscapes":
    from config_cityscapes import Config
    config = Config()
elif dataset == "taskonomy":
    from config_taskonomy import Config
    config = Config()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
net = SceneNet(config.TASKS_NUM_CLASS, config.BACKBONE_NAME).to(device)

In [ ]:
orgin_dict = torch.load("path/to/pretrained_model.pth")
# # If the base_model is multi card trained. Otherwise, comment out the content.
# modified_state_dict = {}
# for key, value in orgin_dict.items():
#     if key.startswith("module."):
#         new_key = key[7:]  # 去掉前7个字符（"module."的长度）
#         modified_state_dict[new_key] = value
#     else:
#         modified_state_dict[key] = value
# net.load_state_dict(modified_state_dict)
net.load_state_dict(orgin_dict)


In [ ]:
parameters_to_prune = []
for name, layer in net.named_modules():
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        if 'backbone' in name or 'task1' in name or 'task3' in name: # task selection
        # if 'backbone' in name or 'task' in name:
            parameters_to_prune.append((layer, 'weight'))

In [ ]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=ration,
)

In [ ]:
def print_sparsity(prune_net, printing=True):
    # Prine the sparsity
    num = 0
    denom = 0
    ct = 0
    for module in prune_net.modules():
        if isinstance(module, nn.modules.conv.Conv2d) or isinstance(module, nn.modules.Linear):
            if hasattr(module, 'weight'):
                num += torch.sum(module.weight == 0)
                denom += module.weight.nelement()
                if printing:
                    print(
                    f"Layer {ct}", "Sparsity in weight: {:.2f}%".format(
                        100. * torch.sum(module.weight == 0) / module.weight.nelement())
                    )
                ct += 1
    if printing:
        print(f"Model Sparsity Now: {num / denom * 100}")
    return num / denom

In [ ]:
print_sparsity(net)

In [ ]:
torch.save(net.state_dict(), f"path/to/save_model.pth")